In [2]:
import numpy as np
import pv_2_20
from pydae.utils import read_data, save_json

In [3]:
M = 2
N = 20
model = pv_2_20.model()
model.ini({},'xy_0.json')
# 'bs_MV0102_MV0101':0.0

True

In [6]:
measurements = {}
conditions = {}
grid_data = read_data('pv_2_20.json')

buse_names = [item['name'] for item in grid_data['buses']]

# Generators active and reactive power (W and var)
# Low Voltage side
for i_m in range(1,M+1):
    for i_n in range(1,N+1):
        name = 'LV' + f"{i_m}".zfill(2) + f"{i_n}".zfill(2)

        S_base = model.get_value(f'S_n_{name}')
        U_base = grid_data['buses'][buse_names.index(name)]['U_kV']*1000

        P = model.get_value(f'p_s_{name}')*S_base
        Q = model.get_value(f'q_s_{name}')*S_base
        U_m = model.get_value(f'V_{name}')*U_base

        S = P + 1j*Q
        S_m = np.abs(S)
        I_m = S_m/(np.sqrt(3)*U_m)


        measurements.update({f'P_{name}':P})
        measurements.update({f'Q_{name}':Q})
        measurements.update({f'U_{name}':U_m}) # "m" means "module"
        measurements.update({f'I_{name}':I_m})    

        
        conditions.update({f'irrad_{name}':model.get_value(f'irrad_{name}')})   


# Feeders
i_n = 1
for i_m in range(1,M+1):
    name = 'POI_MV_MV' + f"{i_m}".zfill(2) + f"{i_n}".zfill(2) 
 
    P_Feeder_pu,Q_Feeder_pu = model.get_mvalue([f'p_line_{name}',f'q_line_{name}'])
    P =-P_Feeder_pu*grid_data['system']['S_base']
    Q =-Q_Feeder_pu*grid_data['system']['S_base']
    S_base = model.get_value(f'S_n_{name}')
    U_base = grid_data['buses'][buse_names.index('POI_MV')]['U_kV']*1000
    U_m = model.get_value(f'V_POI_MV')*U_base
    S = P + 1j*Q
    S_m = np.abs(S)
    I_m = S_m/(np.sqrt(3)*U_m)


    measurements.update({f'P_{name}':P})
    measurements.update({f'Q_{name}':Q})
    measurements.update({f'U_{name}':U_m}) # "m" means "module"
    measurements.update({f'I_{name}':I_m})  


# Powers and voltages at POI
P_POI_pu,Q_POI_pu = model.get_mvalue([f'p_line_POI_GRID',f'q_line_POI_GRID'])
P = P_POI_pu*grid_data['system']['S_base']
Q = Q_POI_pu*grid_data['system']['S_base']
S_base = model.get_value(f'S_n_{name}')
U_base = grid_data['buses'][buse_names.index('POI')]['U_kV']*1000
U_m = model.get_value(f'V_POI')*U_base
S = P + 1j*Q
S_m = np.abs(S)
I_m = S_m/(np.sqrt(3)*U_m)


measurements.update({f'P_POI':P})
measurements.update({f'Q_POI':Q})
measurements.update({f'U_POI':U_m}) # "m" means "module"
measurements.update({f'I_POI':I_m})  

save_json(measurements,file='measurements.json')
save_json(conditions,file='conditions.json')

In [7]:
measurements

{'P_LV0101': 749825.0000000005,
 'Q_LV0101': 1.6971894054167888e-09,
 'U_LV0101': 407.20525510820715,
 'I_LV0101': 1063.1288770575254,
 'P_LV0102': 749824.9999999997,
 'Q_LV0102': -3.592401461256353e-09,
 'U_LV0102': 409.7467197340146,
 'I_LV0102': 1056.534792703487,
 'P_LV0103': 749825.0000000003,
 'Q_LV0103': 2.5662529533865447e-10,
 'U_LV0103': 412.2680392638282,
 'I_LV0103': 1050.0733124210785,
 'P_LV0104': 749825.0000000005,
 'Q_LV0104': 3.1193348935031627e-09,
 'U_LV0104': 414.76910467347034,
 'I_LV0104': 1043.7413508316247,
 'P_LV0105': 749825.0000000002,
 'Q_LV0105': -5.953190783271816e-10,
 'U_LV0105': 417.24981086817013,
 'I_LV0105': 1037.5359180974908,
 'P_LV0106': 749825.0000000001,
 'Q_LV0106': 2.024022290677392e-09,
 'U_LV0106': 419.7100566235196,
 'I_LV0106': 1031.454116391196,
 'P_LV0107': 749824.9999999998,
 'Q_LV0107': -3.0288948567562555e-09,
 'U_LV0107': 422.14974453119777,
 'I_LV0107': 1025.4931365074365,
 'P_LV0108': 749824.9999999995,
 'Q_LV0108': -1.654342803205

In [5]:
model.get_value('bs_MV0102_MV0101')

0.00024

In [6]:
model.report_z()

V_POI_MV =  0.99
V_POI =  1.00
V_GRID =  1.00
V_BESS =  0.99
V_LV0101 =  1.01
V_MV0101 =  1.00
V_LV0102 =  1.01
V_MV0102 =  1.01
V_LV0103 =  1.02
V_MV0103 =  1.01
V_LV0201 =  1.01
V_MV0201 =  1.00
V_LV0202 =  1.01
V_MV0202 =  1.01
V_LV0203 =  1.02
V_MV0203 =  1.01
p_line_POI_GRID =  0.04
q_line_POI_GRID = -0.00
p_line_GRID_POI = -0.04
q_line_GRID_POI =  0.00
p_line_BESS_POI_MV = -0.00
q_line_BESS_POI_MV =  0.00
p_line_POI_MV_BESS =  0.00
q_line_POI_MV_BESS =  0.00
p_line_MV0101_POI_MV =  0.02
q_line_MV0101_POI_MV = -0.00
p_line_POI_MV_MV0101 = -0.02
q_line_POI_MV_MV0101 =  0.00
p_line_MV0201_POI_MV =  0.02
q_line_MV0201_POI_MV = -0.00
p_line_POI_MV_MV0201 = -0.02
q_line_POI_MV_MV0201 =  0.00
p_loss_BESS =  0.00
i_s_BESS =  0.00
e_BESS =  1.15
i_dc_BESS = -0.00
p_s_BESS = -0.00
q_s_BESS =  0.00
alpha_GRID =  0.00
Dv_GRID = -0.00
m_ref_LV0101 =  0.68
v_sd_LV0101 =  0.00
v_sq_LV0101 =  1.01
lvrt_LV0101 =  0.00
m_ref_LV0102 =  0.68
v_sd_LV0102 =  0.00
v_sq_LV0102 =  1.01
lvrt_LV0102 =  0.0

1000.0

In [8]:
Efficiency = 0.15
Irradiance = 1000 # W/m**2
P_n = 1e6

Area_per_PV = P_n/(Irradiance*Efficiency) 
Side_per_PV = np.sqrt(Area_per_PV)
Side_per_PV

81.64965809277261

In [9]:
120*7*20e3*np.sqrt(3)/1e6

29.098453567157136